In [1]:
!pip install torch torchvision torchaudio
!pip install transformers datasets scikit-learn pandas numpy

   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
    --------------------------------------- 2.1/110.9 MB 12.0 MB/s eta 0:00:10
   - -------------------------------------- 3.9/110.9 MB 10.5 MB/s eta 0:00:11
   - -------------------------------------- 5.2/110.9 MB 9.1 MB/s eta 0:00:12
   -- ------------------------------------- 7.1/110.9 MB 8.9 MB/s eta 0:00:12
   -- ------------------------------------- 7.9/110.9 MB 7.7 MB/s eta 0:00:14
   --- ------------------------------------ 8.7/110.9 MB 7.1 MB/s eta 0:00:15
   --- ------------------------------------ 9.7/110.9 MB 6.6 MB/s eta 0:00:16
   --- ------------------------------------ 10.5/110.9 MB 6.3 MB/s eta 0:00:16
   ---- ----------------------------------- 11.3/110.9 MB 6.0 MB/s eta 0:00:17
   ---- ----------------------------------- 12.1/110.9 MB 5.8 MB/s eta 0:00:17
   ---- ----------------------------------- 12.3/110.9 MB 5.5 MB/s eta 0:00:19
   ---- ----------------------------------- 12.8/110.9 MB 5.2 MB/


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.0 MB 4.0 MB/s eta 0:00:03
   -------- ------------------------------- 2.6/12.0 MB 7.1 MB/s eta 0:00:02
   ------------- -------------------------- 4.2/12.0 MB 7.5 MB/s eta 0:00:02
   ----------------- ---------------------- 5.2/12.0 MB 6.7 MB/s eta 0:00:02
   -------------------- ------------------- 6.3/12.0 MB 6.1 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.0 MB 5.8 MB/s eta 0:00:01
   --------------------------- ------------ 8.1/12.0 MB 5.6 MB/s eta 0:00:01
   ------------------------------ --------- 9.2/12.0 MB 5.4 MB/s eta 0:00:01
   --------------------------------- ------ 10.0/12.0 MB 5.3 MB/s eta 0:00:01
   ------------------------------------ --- 11.0/12.0 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------  11.8/12.0 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 5.0 MB/s eta 0:00:00
  


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
